## Datarensning


In [13]:
# This will handle loading and cleaning the Excel data for brukare and medarbetare.

import pandas as pd

def load_data(file_path):
    """
    Load Excel data and return a dictionary of dataframes.
    """
    # Load the Excel file from the given file path
    excel_data = pd.ExcelFile(file_path)
    
    # Parse the sheets into separate dataframes
    data = {
        'brukare': excel_data.parse('Individer, brukare', header=1),
        'medarbetare': excel_data.parse('Medarbetare', header=2)
    }

    return data


def clean_data(brukare_df, medarbetare_df):
    """
    Clean and structure brukare and medarbetare data for analysis.
    """
    # Rename 'Unnamed: 0' to 'Individ' for brukare_df
    brukare_df.rename(columns={'Unnamed: 0': 'Individ'}, inplace=True)

    # Rename 'Unnamed: 0' to 'Medarbetare' for medarbetare_df
    medarbetare_df.rename(columns={'Unnamed: 0': 'Medarbetare'}, inplace=True)

    # Remove brukare with missing or invalid 'Individ' or 'Adress'
    brukare_df = brukare_df.dropna(subset=['Individ', 'Adress']).copy()
    brukare_df = brukare_df[brukare_df['Individ'] != '-']  # Remove rows where 'Individ' is '-'
    brukare_df = brukare_df[brukare_df['Adress'] != '-']  # Remove rows where 'Adress' is '-'
    
    # Fill missing values in other columns with placeholder '-'
    brukare_df.fillna('-', inplace=True)
    medarbetare_df.fillna('-', inplace=True)

    # Convert relevant columns to boolean
    #brukare_df['Kräver körkort'] = brukare_df['Kräver körkort'].apply(lambda x: x == 'Ja')
    #medarbetare_df['Körkort'] = medarbetare_df['Körkort'].apply(lambda x: x == 'Ja')

    columns_brukare = list(brukare_df)
    columns_medarbetare = list(medarbetare_df)

    for column in columns_brukare[1:]:
        brukare_df[column] = brukare_df[column].apply(lambda x: x != '-')

    for column in columns_medarbetare[1:]:
        medarbetare_df[column] = medarbetare_df[column].apply(lambda x: x == 'Ja')


    return brukare_df, medarbetare_df


data = load_data("Studentuppgift fiktiv planering.xlsx")

brukare_df, medarbetare_df = clean_data(data["brukare"], data["medarbetare"])

display(brukare_df)
display(medarbetare_df)

,Individ,Kräver körkort,Behöver läkemedel,Behöver insulin,Har stomi,Röker,Har hund,Har katt,Kräver man,Kräver kvinna,...,Morgon,Förmiddag,Lunch,Eftermiddag,Middag,Tidig kväll,Sen kväll,Dusch,Aktivering,Adress
12,Individ 13,True,True,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,False,False,True
17,Individ 18,True,False,False,False,False,False,False,False,False,...,True,False,True,True,True,False,True,False,False,True
19,Individ 20,False,True,False,False,False,False,True,False,False,...,True,False,False,True,False,False,True,False,False,True
20,Individ 21,False,True,False,False,False,False,False,False,False,...,True,False,False,True,False,False,True,False,False,True
27,Individ 28,True,False,False,False,False,False,True,False,False,...,True,False,False,True,False,True,False,True,False,True
28,Individ 29,True,False,False,False,False,False,False,False,False,...,True,False,False,True,False,False,False,True,False,True
35,Individ 36,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,True
36,Individ 37,True,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,True
37,Individ 38,True,True,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,True
38,Individ 39,True,True,False,False,True,False,False,False,False,...,True,False,False,False,False,False,True,False,False,True


,Medarbetare,Tål hund,Tål katt,Tål rök,Man,Kvinna,Körkort,Läkemedelsdelegering,Insulindelegering,Stomidelegering,18 år el mer
0,Medarbetare 1,True,True,True,False,True,True,False,False,False,True
1,Medarbetare 2,True,True,True,True,False,False,False,False,False,False
2,Medarbetare 3,True,True,True,True,False,True,True,True,True,True
3,Medarbetare 4,True,True,True,False,True,True,True,True,True,True
4,Medarbetare 5,True,True,True,False,True,True,True,True,True,True
5,Medarbetare 6,True,True,True,True,False,True,True,False,False,True
6,Medarbetare 7,True,True,True,False,True,True,True,True,True,True
7,Medarbetare 8,True,True,True,False,True,True,True,True,True,True
8,Medarbetare 9,True,True,True,True,False,True,True,True,True,True
9,Medarbetare 10,True,True,False,False,True,True,True,True,True,True
